<a href="https://colab.research.google.com/github/saba6099/TensorFlow-Projects/blob/master/Transfer_Learning_and_Fine_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Stage 1: Install dependencies and setting up GPU environment

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 102kB/s 
     |████████████████████████████████| 3.0MB 18.3MB/s 
     |████████████████████████████████| 419kB 44.2MB/s 


In [2]:
!pip install tqdm

### Downloading the Dogs vs Cats dataset 

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2019-10-12 11:17:02--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   145MB/s    in 0.5s    

2019-10-12 11:17:03 (145 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Stage 2: Dataset preprocessing

### Import project dependencies

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-alpha0'

### Unzipping the Dogs vs Cats dataset

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Seting up dataset paths

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [0]:
IMG_SHAPE = (128, 128, 3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [0]:
base_model.summary()

### Freezing the base model

In [0]:
base_model.trainable = False

### Defining the custom head for our network

In [26]:
base_model.output

<tf.Tensor 'out_relu_1/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [28]:
global_average_layer

<tf.Tensor 'global_average_pooling2d_1/Mean:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [0]:
# model.summary()

### Compiling the model

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [34]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [35]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [36]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 63s 4s/step - loss: 0.2280 - accuracy: 0.9040 - val_loss: 0.0998 - val_accuracy: 0.9610
Epoch 2/5
16/16 [==============================] - 12s 768ms/step - loss: 0.0236 - accuracy: 0.9985 - val_loss: 0.0952 - val_accuracy: 0.9600
Epoch 3/5
16/16 [==============================] - 12s 770ms/step - loss: 0.0107 - accuracy: 0.9995 - val_loss: 0.1024 - val_accuracy: 0.9620
Epoch 4/5
16/16 [==============================] - 12s 759ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.1111 - val_accuracy: 0.9630
Epoch 5/5
16/16 [==============================] - 12s 753ms/step - loss: 9.8356e-04 - accuracy: 1.0000 - val_loss: 0.1079 - val_accuracy: 0.9650


### Transfer learning model evaluation

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [38]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9649999737739563


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [0]:
base_model.trainable = True

In [42]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 155


In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [46]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 34s 2s/step - loss: 8.9026e-04 - accuracy: 1.0000 - val_loss: 0.1948 - val_accuracy: 0.9560
Epoch 2/5
16/16 [==============================] - 9s 557ms/step - loss: 1.8876e-04 - accuracy: 1.0000 - val_loss: 0.1581 - val_accuracy: 0.9650
Epoch 3/5
16/16 [==============================] - 9s 557ms/step - loss: 5.6619e-05 - accuracy: 1.0000 - val_loss: 0.1671 - val_accuracy: 0.9650
Epoch 4/5
16/16 [==============================] - 9s 538ms/step - loss: 2.3213e-04 - accuracy: 1.0000 - val_loss: 0.1440 - val_accuracy: 0.9680
Epoch 5/5
16/16 [==============================] - 9s 541ms/step - loss: 4.0051e-05 - accuracy: 1.0000 - val_loss: 0.1451 - val_accuracy: 0.9670


### Evaluating the fine tuned model

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [48]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9670000076293945
